In [15]:
import pandas as pd
from pathlib import Path
import os

In [20]:
folder_path = "D:/Code_ML/energy_consumption_forecasting/src/data/raw"

print("Folder exists:", os.path.exists(folder_path))
print("Files in folder:", os.listdir(folder_path))
print("CSV files found:", glob.glob(os.path.join(folder_path, "*.csv")))

Folder exists: True
Files in folder: ['AEP_hourly.csv', 'DAYTON_hourly.csv', 'DOM_hourly.csv', 'DUQ_hourly.csv']
CSV files found: ['D:/Code_ML/energy_consumption_forecasting/src/data/raw\\AEP_hourly.csv', 'D:/Code_ML/energy_consumption_forecasting/src/data/raw\\DAYTON_hourly.csv', 'D:/Code_ML/energy_consumption_forecasting/src/data/raw\\DOM_hourly.csv', 'D:/Code_ML/energy_consumption_forecasting/src/data/raw\\DUQ_hourly.csv']


In [ ]:

folder_path = "D:/Code_ML/energy_consumption_forecasting/src/data/raw"  # change this to your folder



              Datetime   AEP_MW  DAYTON_MW  DOM_MW  DUQ_MW
0  2004-12-31 01:00:00  13478.0        NaN     NaN     NaN
1  2004-12-31 02:00:00  12865.0        NaN     NaN     NaN
2  2004-12-31 03:00:00  12577.0        NaN     NaN     NaN
3  2004-12-31 04:00:00  12517.0        NaN     NaN     NaN
4  2004-12-31 05:00:00  12670.0        NaN     NaN     NaN


In [25]:

data_dir = Path(folder_path)
for csv_file in data_dir.glob('*.csv'):
    print(csv_file)

D:\Code_ML\energy_consumption_forecasting\src\data\raw\AEP_hourly.csv
D:\Code_ML\energy_consumption_forecasting\src\data\raw\DAYTON_hourly.csv
D:\Code_ML\energy_consumption_forecasting\src\data\raw\DOM_hourly.csv
D:\Code_ML\energy_consumption_forecasting\src\data\raw\DUQ_hourly.csv


In [ ]:
def load_raw_data(data_dir):
    '''
    Load all raw energy consumption CSV files and return a merged Dataframe
    '''

    data_dir = Path(folder_path)

    dataframes = []

    for csv_file in data_dir.glob('*.csv'):

        print(csv_file)

        df = pd.read_csv('csv_file')

        # Basic Validation

        if 'Datetime' not in df.columns or 'MW' not in df.columns:
            raise ValueError(f'Missing required columns in {csv_file.name}') # type: ignore
        
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        df['utility'] = csv_file.stem.replace('_hourly','') # type: ignore

        dataframes.append(df)

    merged_df = pd.concat(dataframes, ignore_index=True)
    merged_df = merged_df.sort_values("Datetime")

    return merged_df

if __name__ == '__main__':

    df = load_raw_data(folder_path)
    print(df.head())
    print(df.info())

AttributeError: 'str' object has no attribute 'glob'

In [ ]:
from pathlib import Path
import pandas as pd

def load_raw_data(folder_path):
    """
    Load all raw energy consumption CSV files and return a merged DataFrame
    """
    data_dir = Path(folder_path)   # 🔑 convert str → Path
    dataframes = []

    for csv_file in data_dir.glob("*.csv"):
        print(csv_file)
        df = pd.read_csv(csv_file)   # 🔑 pass the variable, not a string
        dataframes.append(df)

    if not dataframes:
        raise FileNotFoundError(f"No CSV files found in {data_dir}")

    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df


if __name__ == "__main__":
    folder_path = "D:/Code_ML/energy_consumption_forecasting/src/data/raw"
    df = load_raw_data(folder_path)

    print(df.head())
    print(df.info())


FileNotFoundError: No CSV files found in D:\Code_ML\senergy_consumption_forecasting\src\data\raw